# <center> Introduction to Spark In-memory Computing via Python PySpark </center>

In [1]:
import sys
import os

sys.path.insert(0, '/usr/hdp/current/spark2-client/python')
sys.path.insert(0, '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip')

os.environ['SPARK_HOME'] = '/usr/hdp/current/spark2-client/'
os.environ['SPARK_CONF_DIR'] = '/etc/spark2/conf'
os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python'

import pyspark
conf = pyspark.SparkConf()
conf.setMaster("yarn")
conf.set("spark.driver.memory","2g")
conf.set("spark.executor.instances", "5")
conf.set("spark.executor.memory","5g")
conf.set("spark.executor.cores","5")

sc = pyspark.SparkContext(conf=conf)
sc

<SparkContext master=yarn appName=pyspark-shell>

### Airlines Data

**Spark SQL**
- Spark module for structured data processing
- provides more information about the structure of both the data and the computation being performed for additional optimization
- execute SQL queries written using either a basic SQL syntax or HiveQL

**DataFrame**
- a distributed collection of data organized into named columns
- conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood
- can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs

In [2]:
sqlContext = pyspark.SQLContext(sc)
sqlContext

In [3]:
!hdfs dfs -ls -h /repository/airlines/data

Found 22 items
-rw-r--r--   3 hdfs hdfs    121.3 M 2019-12-30 11:28 /repository/airlines/data/1987.csv
-rw-r--r--   3 hdfs hdfs    477.8 M 2019-12-30 11:28 /repository/airlines/data/1988.csv
-rw-r--r--   3 hdfs hdfs    464.0 M 2019-12-30 11:28 /repository/airlines/data/1989.csv
-rw-r--r--   3 hdfs hdfs    485.6 M 2019-12-30 11:29 /repository/airlines/data/1990.csv
-rw-r--r--   3 hdfs hdfs    468.5 M 2019-12-30 11:28 /repository/airlines/data/1991.csv
-rw-r--r--   3 hdfs hdfs    469.5 M 2019-12-30 11:27 /repository/airlines/data/1992.csv
-rw-r--r--   3 hdfs hdfs    468.0 M 2019-12-30 11:29 /repository/airlines/data/1993.csv
-rw-r--r--   3 hdfs hdfs    478.3 M 2019-12-30 11:29 /repository/airlines/data/1994.csv
-rw-r--r--   3 hdfs hdfs    506.2 M 2019-12-30 11:28 /repository/airlines/data/1995.csv
-rw-r--r--   3 hdfs hdfs    509.2 M 2019-12-30 11:29 /repository/airlines/data/1996.csv
-rw-r--r--   3 hdfs hdfs    515.3 M 2019-12-30 11:29 /repository/airlines/data/1997.csv
-rw-r--r--   3 hd

In [4]:
airlines = sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load("/repository/airlines/data/1987.csv")\
    .cache()

In [5]:
%%time
airlines.count()

CPU times: user 2.17 ms, sys: 211 µs, total: 2.38 ms
Wall time: 2.62 s


1311826

In [6]:
%%time
airlines.count()

CPU times: user 1.69 ms, sys: 0 ns, total: 1.69 ms
Wall time: 376 ms


1311826

In [7]:
airlines.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

You can interact with a DataFrame via SQLContext using SQL statements by registering the DataFrame as a table

In [16]:
airlines.registerTempTable("airlines")

*How many unique airlines are there?*

In [17]:
uniqueAirline = sqlContext.sql("SELECT DISTINCT UniqueCarrier \
                                FROM airlines")
uniqueAirline.show()

+-------------+
|UniqueCarrier|
+-------------+
|           UA|
|           EA|
|           PI|
|           PS|
|           AA|
|           NW|
|           HP|
|           TW|
|           DL|
|           US|
|           AS|
|           CO|
|       PA (1)|
|           WN|
+-------------+



*Calculate how many flights completed by each carrier over time*

In [19]:
%%time
carrierFlightCount = sqlContext.sql("SELECT UniqueCarrier, COUNT(UniqueCarrier) AS FlightCount \
                                    FROM airlines GROUP BY UniqueCarrier")
carrierFlightCount.show()

+-------------+-----------+
|UniqueCarrier|FlightCount|
+-------------+-----------+
|           UA|     152624|
|           EA|     108776|
|           PI|     116482|
|           PS|      41706|
|           AA|     165121|
|           NW|     108273|
|           HP|      45399|
|           TW|      69650|
|           DL|     185813|
|           US|      94814|
|           AS|      21406|
|           CO|     123002|
|       PA (1)|      16785|
|           WN|      61975|
+-------------+-----------+

CPU times: user 741 µs, sys: 1.47 ms, total: 2.21 ms
Wall time: 402 ms


*How do you display full carrier names?*

In [21]:
carriers = sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load("/repository/airlines/metadata/carriers.csv")\
    .cache()
carriers.registerTempTable("carriers")

In [22]:
carriers.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)



In [23]:
%%time
carrierFlightCountFullName = sqlContext.sql("SELECT c.Description, a.UniqueCarrier, COUNT(a.UniqueCarrier) AS FlightCount \
                                    FROM airlines AS a \
                                    INNER JOIN carriers AS c \
                                    ON c.Code = a.UniqueCarrier \
                                    GROUP BY a.UniqueCarrier, c.Description \
                                    ORDER BY a.UniqueCarrier")
carrierFlightCountFullName.show()

+--------------------+-------------+-----------+
|         Description|UniqueCarrier|FlightCount|
+--------------------+-------------+-----------+
|American Airlines...|           AA|     165121|
|Alaska Airlines Inc.|           AS|      21406|
|Continental Air L...|           CO|     123002|
|Delta Air Lines Inc.|           DL|     185813|
|Eastern Air Lines...|           EA|     108776|
|America West Airl...|           HP|      45399|
|Northwest Airline...|           NW|     108273|
|Pan American Worl...|       PA (1)|      16785|
|Piedmont Aviation...|           PI|     116482|
|Pacific Southwest...|           PS|      41706|
|Trans World Airwa...|           TW|      69650|
|United Air Lines ...|           UA|     152624|
|US Airways Inc. (...|           US|      94814|
|Southwest Airline...|           WN|      61975|
+--------------------+-------------+-----------+

CPU times: user 2.02 ms, sys: 49 µs, total: 2.07 ms
Wall time: 1.33 s


*What is the averaged departure delay time for each airline?*

In [24]:
%%time
avgDepartureDelay = sqlContext.sql("SELECT FIRST(c.Description), FIRST(a.UniqueCarrier), AVG(a.DepDelay) AS AvgDepDelay \
                                    FROM airlines AS a \
                                    INNER JOIN carriers AS c \
                                    ON c.Code = a.UniqueCarrier \
                                    GROUP BY a.UniqueCarrier \
                                    ORDER BY a.UniqueCarrier")
avgDepartureDelay.show()

+-------------------------+---------------------------+------------------+
|first(Description, false)|first(UniqueCarrier, false)|       AvgDepDelay|
+-------------------------+---------------------------+------------------+
|     American Airlines...|                         AA| 5.931756197713971|
|     Alaska Airlines Inc.|                         AS| 8.302911613420791|
|     Continental Air L...|                         CO|12.620661609706707|
|     Delta Air Lines Inc.|                         DL|7.3701212477415705|
|     Eastern Air Lines...|                         EA| 8.832762739602341|
|     America West Airl...|                         HP| 6.196152731649689|
|     Northwest Airline...|                         NW| 6.856916558873704|
|     Pan American Worl...|                     PA (1)| 4.951404799228265|
|     Piedmont Aviation...|                         PI| 7.724536454582527|
|     Pacific Southwest...|                         PS| 13.31959242658045|
|     Trans World Airwa..

In [ ]:
airlines.unpersist()

In [ ]:
sc.stop()